In [ ]:
# Create venv
!python -m venv /content/taenv
!/content/taenv/bin/python -m pip -q install -U pip setuptools wheel

# Core numeric stack (stable, ABI-safe)
!/content/taenv/bin/pip -q install numpy==1.26.4 pandas==2.2.2 scipy==1.11.4 scikit-learn==1.4.2

# Torch (CPU is fine for these augmenters)
!/content/taenv/bin/pip -q install torch==2.3.1

# spaCy + small English model
!/content/taenv/bin/pip -q install spacy==3.8.7
!/content/taenv/bin/python -m spacy download en_core_web_sm

# HF stack (pin datasets 2.x to avoid TextAttack conflicts)
!/content/taenv/bin/pip -q install transformers==4.44.2 tokenizers==0.19.1 accelerate==0.30.1 \
                         sentencepiece==0.2.0 sacremoses==0.1.1 datasets==2.18.0

# TextAttack and helpers (NO Flair needed)
!/content/taenv/bin/pip -q install textattack==0.3.8 tqdm==4.66.4 xlsxwriter==3.2.0

# NLTK (for WordNet); fetch corpora
!/content/taenv/bin/pip -q install nltk==3.9.1
!/content/taenv/bin/python -c "import nltk; nltk.download('wordnet'); nltk.download('omw-1.4'); print('NLTK ready')"

# (Optional) for --sim_gating
!/content/taenv/bin/pip -q install sentence-transformers==3.0.1


Error: Command '['/content/taenv/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/content/taenv/bin/python: No module named pip
/bin/bash: line 1: /content/taenv/bin/pip: No such file or directory
/bin/bash: line 1: /content/taenv/bin/pip: No such file or directory
/bin/bash: line 1: /content/taenv/bin/pip: No such file or directory
/content/taenv/bin/python: No module named spacy
/bin/bash: line 1: /content/taenv/bin/pip: No such file or directory
/bin/bash: line 1: /content/taenv/bin/pip: No such file or directory
/bin/bash: line 1: /content/taenv/bin/pip: No such file or directory
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'nltk'
/bin/bash: line 1: /content/taenv/bin/pip: No such file or directory


In [ ]:
%%writefile /content/augment6.py
# <-- paste the 6-augmenter script I gave (no BackTranslation, no CLARE, skips `context`) -->


Writing /content/augment6.py


In [ ]:
!/content/taenv/bin/python /content/augment6.py \
  --input /content/advanced_adversarial_prompts.csv \
  --run_semantics --run_lexical --run_entity \
  --n_semantic 2 --n_lexical 2 --n_entity 2 \
  --write_wide


In [ ]:
# ============================================
# Adversarial Prompt Augmentation (6 augmenters, no BT/CLARE)
# - Uses: EasyDataAugmenter, WordNetAugmenter, EmbeddingAugmenter,
#         CharSwapAugmenter, CheckListAugmenter, Entity-swap
# - Skips 'context' column
# - Auto-enables groups if none passed
# - Optional semantic similarity gating (--sim_gating)
# ============================================
import os, re, json, random, uuid, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

# ---- spaCy for entity-centric swaps (self-healing load)
import spacy
try:
    nlp = spacy.load("en_core_web_sm")
except Exception:
    try:
        from spacy.cli import download
        download("en_core_web_sm")
        nlp = spacy.load("en_core_web_sm")
    except Exception as e:
        raise RuntimeError(f"Failed to load/download spaCy 'en_core_web_sm': {e}")

# ---- Optional semantic similarity gating (off by default)
_ST_MODEL = None
def _lazy_load_st():
    global _ST_MODEL
    if _ST_MODEL is None:
        try:
            from sentence_transformers import SentenceTransformer
            _ST_MODEL = SentenceTransformer("all-MiniLM-L6-v2")
        except Exception as e:
            print(f"⚠️ sentence-transformers not available for sim gating: {e}")
            _ST_MODEL = False
    return _ST_MODEL

# ---- TextAttack augmenters (0.3.8)
from textattack.augmentation import (
    EasyDataAugmenter,
    WordNetAugmenter,
    EmbeddingAugmenter,
    CharSwapAugmenter,
    CheckListAugmenter,
)

# ============================
# Defaults
# ============================
DEFAULT_INPUT = "advanced_adversarial_prompts.csv"
DEFAULT_OUTPUT_LONG = "prompts_adversarial_augmented_long.csv"
DEFAULT_OUTPUT_WIDE = "prompts_adversarial_augmented_wide.xlsx"

# Only these columns get augmented (explicitly skip 'context')
SOURCE_COLUMNS = [
    "original_prompt", "amplified", "paradoxical",
    "ambiguous", "entity_mixup", "cot_escalation"
]
SKIP_COLUMNS = {"context"}

# CoT splitting
COT_SPLIT_REGEX = r"\s*\|\|\|\s*|\s*\|\|\s*|###|\n{2,}"

# ============================
# Entity pools (neutral)
# ============================
ENTITY_POOLS = {
    "PERSON_FIRST": [
        "Alex","Jordan","Taylor","Casey","Riley","Sam","Avery","Drew","Cameron","Morgan",
        "Jamie","Quinn","Parker","Sydney","Rowan"
    ],
    "PERSON_LAST": [
        "Smith","Patel","Garcia","Lee","Khan","Johnson","Kim","Singh","Nguyen","Hernandez",
        "Brown","Davis","Martinez","Wilson","Anderson"
    ],
    "ORG": [
        "Northbridge Analytics","Orion Labs","Cedar Systems","Atlas Research",
        "Summit Media","BlueRiver Health","Vertex Finance","Nimbus AI"
    ],
    "GPE": ["Canada","India","Brazil","Japan","Germany","Kenya","Spain","UAE","Australia","Mexico"],
    "NORP": ["engineers","artists","journalists","researchers","students","teachers",
             "nurses","developers","lawyers","scientists"],
}
def _rand_person():
    return f"{random.choice(ENTITY_POOLS['PERSON_FIRST'])} {random.choice(ENTITY_POOLS['PERSON_LAST'])}"

# ============================
# Augmenter factories (6 total)
# ============================
def build_semantics_augmenters():
    # (1) EDA (light paraphrasing: insertion/deletion/swap/synonym)
    # (2) WordNet synonyms
    return [
        ("semantics.eda", EasyDataAugmenter(pct_words_to_swap=0.1, transformations_per_example=8)),
        ("semantics.wordnet", WordNetAugmenter(pct_words_to_swap=0.2)),
    ]

def build_lexical_char_augmenters():
    # (3) Embedding swaps
    # (4) Character swaps (DeepWordBug-like)
    # (5) CheckList invariances (case/punct/etc.)
    return [
        ("lexical.embedding", EmbeddingAugmenter(pct_words_to_swap=0.2)),
        ("char.charswap", CharSwapAugmenter()),
        ("lexical.checklist", CheckListAugmenter()),
    ]

# (6) Entity-centric swap helper
def entity_swap_variants(text: str, n: int = 2):
    doc = nlp(text)
    variants = []
    for _ in range(n):
        rep = {}
        for ent in doc.ents:
            if ent.label_ == "PERSON":
                rep[ent.text] = _rand_person()
            elif ent.label_ == "ORG":
                rep[ent.text] = random.choice(ENTITY_POOLS["ORG"])
            elif ent.label_ == "GPE":
                rep[ent.text] = random.choice(ENTITY_POOLS["GPE"])
            elif ent.label_ == "NORP":
                rep[ent.text] = random.choice(ENTITY_POOLS["NORP"])
        new_text = text
        if rep:
            # replace longest-first to avoid partial overlaps
            for k in sorted(rep.keys(), key=len, reverse=True):
                new_text = re.sub(rf"\b{re.escape(k)}\b", rep[k], new_text)
            variants.append(new_text)
        else:
            variants.append(f"{_rand_person()} mentioned that {text}")
    # dedup
    seen, uniq = set(), []
    for v in variants:
        v2 = v.strip()
        if v2 and v2 not in seen:
            seen.add(v2); uniq.append(v2)
    return uniq[:n]

# ============================
# Helpers
# ============================
def split_cot_field(value):
    if pd.isna(value):
        return []
    s = str(value).strip()
    if s.startswith("["):
        try:
            arr = json.loads(s)
            return [str(x).strip() for x in arr if str(x).strip()]
        except Exception:
            pass
    parts = re.split(COT_SPLIT_REGEX, s)
    parts = [p.strip() for p in parts if p and p.strip()]
    if len(parts) == 1 and "\n" in s:
        lines = [ln.strip() for ln in s.split("\n") if ln.strip()]
        if 2 <= len(lines) <= 5:
            parts = lines
    return parts

def passes_basic_gates(original: str, augmented: str, min_tokens: int) -> bool:
    if not augmented:
        return False
    o = original.strip()
    a = augmented.strip()
    if not a or a.lower() == o.lower():
        return False
    if len(a.split()) < min_tokens:
        return False
    return True

def sim_gate_ok(original: str, augmented: str, sim_thresh: float, cache: dict) -> bool:
    if sim_thresh is None:
        return True
    model = _lazy_load_st()
    if not model:
        return True
    if original not in cache:
        cache[original] = model.encode([original], normalize_embeddings=True)[0]
    if augmented not in cache:
        cache[augmented] = model.encode([augmented], normalize_embeddings=True)[0]
    v1, v2 = cache[original], cache[augmented]
    return float(np.dot(v1, v2)) >= sim_thresh

def safe_augment(augmenter, text: str, k: int):
    try:
        out = augmenter.augment(text)
        if not out:
            return []
        uniq = list(dict.fromkeys([o.strip() for o in out if isinstance(o, str) and o.strip()]))
        random.shuffle(uniq)
        return uniq[:k]
    except Exception:
        return []

# ============================
# Optional wide writer
# ============================
def write_wide_xlsx(long_df: pd.DataFrame, out_path: str, source_cols: list):
    key_cols = ["source_row_idx", "source_column", "augmented_text"]
    subset = long_df[key_cols].copy()
    agg = subset.groupby(["source_row_idx", "source_column"])["augmented_text"] \
                .apply(lambda xs: "\n".join(list(dict.fromkeys(xs)))) \
                .reset_index()
    rows = []
    max_row = int(long_df["source_row_idx"].max()) if not long_df.empty else -1
    for ridx in range(max_row + 1):
        row_dict = {"row_index": ridx}
        for col in source_cols:
            val = agg[(agg["source_row_idx"] == ridx) & (agg["source_column"] == col)]
            row_dict[col] = "" if val.empty else val["augmented_text"].values[0]
        rows.append(row_dict)
    wide_df = pd.DataFrame(rows)
    with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
        wide_df.to_excel(writer, index=False, sheet_name="augmented")
    return wide_df

# ============================
# Main
# ============================
def main():
    ap = argparse.ArgumentParser(description="Adversarial augmentation (6 augmenters, no BT/CLARE).")
    ap.add_argument("--input", default=DEFAULT_INPUT, help="Input CSV file")
    ap.add_argument("--output_long", default=DEFAULT_OUTPUT_LONG, help="Output long CSV filename")
    ap.add_argument("--output_wide", default=DEFAULT_OUTPUT_WIDE, help="Output wide XLSX filename")
    ap.add_argument("--seed", type=int, default=42, help="Random seed")

    # Which groups to run
    ap.add_argument("--run_semantics", action="store_true",
                    help="Enable semantics-preserving group (EDA + WordNet)")
    ap.add_argument("--run_lexical", action="store_true",
                    help="Enable lexical/char group (Embedding/CharSwap/CheckList)")
    ap.add_argument("--run_entity", action="store_true",
                    help="Enable entity-centric group (NER swaps)")

    # Per-augmenter caps
    ap.add_argument("--n_semantic", type=int, default=2, help="Samples per semantic augmenter")
    ap.add_argument("--n_lexical", type=int, default=2, help="Samples per lexical/char augmenter")
    ap.add_argument("--n_entity", type=int, default=2, help="Entity swap variants per text")

    # Gates
    ap.add_argument("--min_tokens", type=int, default=5, help="Minimum tokens in augmented text")

    # Optional similarity gating
    ap.add_argument("--sim_gating", action="store_true", help="Enable semantic similarity gating")
    ap.add_argument("--sim_thresh", type=float, default=0.85, help="Cosine threshold if --sim_gating")

    # Optional wide
    ap.add_argument("--write_wide", action="store_true", help="Write wide XLSX summary")

    # For notebook: parse empty list; for CLI: change to ap.parse_args()
    args = ap.parse_args([])

    # Auto-enable all groups if none were specified
    if not (args.run_semantics or args.run_lexical or args.run_entity):
        args.run_semantics = args.run_lexical = args.run_entity = True

    # Seed
    random.seed(args.seed)
    np.random.seed(args.seed)

    # Read CSV with encoding fallback
    try:
        df = pd.read_csv(args.input)
    except UnicodeDecodeError:
        df = pd.read_csv(args.input, encoding="latin1")

    present = [c for c in SOURCE_COLUMNS if c in df.columns]
    cols = [c for c in present if c not in SKIP_COLUMNS]
    if not cols:
        raise ValueError(f"No expected columns found in {args.input}. Got: {list(df.columns)}")

    print("🔎 Found columns:", list(df.columns))
    print("✅ Will augment:", cols)
    print("➕ Active groups:",
          "semantics" if args.run_semantics else "-",
          "lexical"   if args.run_lexical   else "-",
          "entity"    if args.run_entity    else "-")

    sem_aug = build_semantics_augmenters() if args.run_semantics else []
    lex_aug = build_lexical_char_augmenters() if args.run_lexical else []
    run_entity = args.run_entity

    sim_cache = {}
    if args.sim_gating:
        _lazy_load_st()

    records = []
    total_cells = 0

    for ridx, row in tqdm(df.iterrows(), total=len(df), desc="Augmenting"):
        for col in cols:
            raw = row[col]
            if pd.isna(raw):
                continue
            texts = split_cot_field(raw) if col == "cot_escalation" else [str(raw)]
            for sub_idx, text in enumerate(texts):
                text = text.strip()
                if not text:
                    continue
                total_cells += 1
                base_id = str(uuid.uuid4())

                # (1) Semantics (EDA + WordNet)
                for name, aug in sem_aug:
                    outs = safe_augment(aug, text, k=args.n_semantic)
                    for out in outs:
                        if not passes_basic_gates(text, out, args.min_tokens):
                            continue
                        if not sim_gate_ok(text, out, args.sim_thresh if args.sim_gating else None, sim_cache):
                            continue
                        records.append({
                            "source_row_idx": ridx,
                            "source_column": col,
                            "cot_index": sub_idx if col == "cot_escalation" else None,
                            "group": "semantics_preserving",
                            "augmenter": name,
                            "original_text": text,
                            "augmented_text": out,
                            "seed": args.seed,
                            "uuid": base_id
                        })

                # (2) Lexical / char
                for name, aug in lex_aug:
                    outs = safe_augment(aug, text, k=args.n_lexical)
                    for out in outs:
                        if not passes_basic_gates(text, out, args.min_tokens):
                            continue
                        if not sim_gate_ok(text, out, args.sim_thresh if args.sim_gating else None, sim_cache):
                            continue
                        records.append({
                            "source_row_idx": ridx,
                            "source_column": col,
                            "cot_index": sub_idx if col == "cot_escalation" else None,
                            "group": "lexical_char",
                            "augmenter": name,
                            "original_text": text,
                            "augmented_text": out,
                            "seed": args.seed,
                            "uuid": base_id
                        })

                # (3) Entity-centric
                if run_entity:
                    outs = entity_swap_variants(text, n=args.n_entity)
                    for out in outs:
                        if not passes_basic_gates(text, out, args.min_tokens):
                            continue
                        if not sim_gate_ok(text, out, args.sim_thresh if args.sim_gating else None, sim_cache):
                            continue
                        records.append({
                            "source_row_idx": ridx,
                            "source_column": col,
                            "cot_index": sub_idx if col == "cot_escalation" else None,
                            "group": "entity_centric",
                            "augmenter": "ner.swap",
                            "original_text": text,
                            "augmented_text": out,
                            "seed": args.seed,
                            "uuid": base_id
                        })

    out_df = pd.DataFrame.from_records(records)
    out_df.to_csv(args.output_long, index=False)
    print(f"✅ Saved {len(out_df):,} augmented rows to: {args.output_long} (from {total_cells} source cells)")

    if args.write_wide and len(out_df):
        write_wide_xlsx(out_df, args.output_wide, cols)
        print(f"🧾 Also wrote wide Excel to: {args.output_wide}")

if __name__ == "__main__":
    main()


🔎 Found columns: ['original_prompt', 'context', 'amplified', 'paradoxical', 'ambiguous', 'entity_mixup', 'cot_escalation']
✅ Will augment: ['original_prompt', 'amplified', 'paradoxical', 'ambiguous', 'entity_mixup', 'cot_escalation']
➕ Active groups: semantics lexical entity


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:10<00:00, 47.4MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpny8l8o2f.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
Augmenting:   0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended

pytorch_model.bin:   0%|          | 0.00/419M [00:00<?, ?B/s]

2025-08-17 13:49:48,492 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


Augmenting: 100%|██████████| 50/50 [1:15:34<00:00, 90.70s/it] 

✅ Saved 2,433 augmented rows to: prompts_adversarial_augmented_long.csv (from 321 source cells)


In [ ]:
pip install textattack.augmentation

ERROR: Could not find a version that satisfies the requirement textattack.augmentation (from versions: none)
ERROR: No matching distribution found for textattack.augmentation


The six augmenters
1) Semantics-preserving (factory build_semantics_augmenters)

EasyDataAugmenter (semantics.eda): small swaps/insertions/deletions/synonym replacements; configured with pct_words_to_swap=0.1, transformations_per_example=8.

WordNetAugmenter (semantics.wordnet): synonym substitutions based on WordNet; pct_words_to_swap=0.2.

2) Lexical / character (factory build_lexical_char_augmenters)

EmbeddingAugmenter (lexical.embedding): replaces words with nearest neighbors in an embedding space; pct_words_to_swap=0.2.

CharSwapAugmenter (char.charswap): character-level operations (insert/delete/substitute/swap) to simulate typos/adversarial character noise.

CheckListAugmenter (lexical.checklist): invariance transforms (e.g., punctuation/casing/contractions) based on CheckList methodology.

The exact low-level behavior follows TextAttack’s implementations; the script treats them as black boxes.

3) Entity-centric swap (entity_swap_variants)

Parses text with spaCy NER once per source text and builds a replacement dict:

PERSON → a random neutral first+last pair (_rand_person()),

ORG → a random org from pool,

GPE → a random geopolitical entity,

NORP → a random demonym/collective (e.g., “engineers”).

Longest-first replacement: sorts keys by length desc and uses \b...\b word-boundary regex. This avoids replacing substrings inside larger names (“Ann” inside “Annabelle”).

If the doc has no entities, it falls back to prefixing the text with a neutral speaker stub:

"Alex Smith mentioned that <original text>" (with a randomized first/last).

Deduplicates variants; returns up to n unique variants.